In [1]:
import json
import boto3
import fsspec
import numpy as np
import xarray as xr
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc
from dash import dcc
import plotly.express as px
from datetime import datetime

In [2]:
sts = boto3.client('sts')

assumed_role = sts.assume_role(
    RoleArn='arn:aws:iam::690817015997:role/nasa-power-s3-access-role',
    RoleSessionName='NasaPowerDataExplorationSession2'  # Provide a session name
)

# Extract temporary credentials from the assumed role response
credentials = assumed_role['Credentials']

In [3]:
aws_access_key_id = credentials['AccessKeyId']
aws_secret_access_key = credentials['SecretAccessKey']
aws_session_token=credentials['SessionToken']

s3_client = boto3.client('s3', aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_access_key, aws_session_token = aws_session_token)

bucket_contents = s3_client.list_objects_v2(Bucket='power-analysis-ready-datastore')
#bucket_contents = s3_client.list_objects_v2(Bucket='nasa-power-combined-data')

#list_of_objs_in_bucket = bucket_contents['Contents']
#print(len(list_of_objs_in_bucket))

In [ ]:
bucket_contents

In [4]:
filepath = 'https://power-analysis-ready-datastore.s3.amazonaws.com/power_901_daily_meteorology_utc.zarr'
filepath_mapped = fsspec.get_mapper(filepath)

ds_daily = xr.open_zarr(store=filepath_mapped, consolidated=True)

/Users/sumangangopadhyay/nasa-power-data-exploration/nasa_power_venv/lib/python3.8/site-packages/xarray/backends/plugins.py:71: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


In [5]:
ds_daily

<xarray.Dataset>
Dimensions:      (time: 16071, lat: 361, lon: 576)
Coordinates:
  * lat          (lat) float64 -90.0 -89.5 -89.0 -88.5 ... 88.5 89.0 89.5 90.0
  * lon          (lon) float64 -180.0 -179.4 -178.8 -178.1 ... 178.1 178.8 179.4
  * time         (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2024-12-31
Data variables: (12/66)
    CDD0         (time, lat, lon) float64 dask.array<chunksize=(14610, 5, 5), meta=np.ndarray>
    CDD10        (time, lat, lon) float64 dask.array<chunksize=(14610, 5, 5), meta=np.ndarray>
    CDD18_3      (time, lat, lon) float64 dask.array<chunksize=(14610, 5, 5), meta=np.ndarray>
    DISPH        (time, lat, lon) float64 dask.array<chunksize=(14610, 5, 5), meta=np.ndarray>
    EVLAND       (time, lat, lon) float64 dask.array<chunksize=(14610, 5, 5), meta=np.ndarray>
    EVPTRNS      (time, lat, lon) float64 dask.array<chunksize=(14610, 5, 5), meta=np.ndarray>
    ...           ...
    WS2M_RANGE   (time, lat, lon) float64 dask.array<chunksize=(14610, 5, 5), meta=np.ndarray>
    WS50M        (time, lat, lon) float64 dask.array<chunksize=(14610, 5, 5), meta=np.ndarray>
    WS50M_MAX    (time, lat, lon) float64 dask.array<chunksize=(14610, 5, 5), meta=np.ndarray>
    WS50M_MIN    (time, lat, lon) float64 dask.array<chunksize=(14610, 5, 5), meta=np.ndarray>
    WS50M_RANGE  (time, lat, lon) float64 dask.array<chunksize=(14610, 5, 5), meta=np.ndarray>
    Z0M          (time, lat, lon) float64 dask.array<chunksize=(14610, 5, 5), meta=np.ndarray>
Attributes: (12/37)
    acknowledgement:            The Prediction of Worldwide Energy Resources ...
    comment:                    Version 9.0.1 includes source data resolution.
    conventions:                CF-1.8, ACDD-1.3
    creator_email:              bradley.macpherson@nasa.gov
    creator_name:               Bradley Macpherson
    date_created:               2021-06-16T10:30:00.000000
    ...                         ...
    time_coverage_end:          2024-12-31T00:00:00
    time_coverage_resolution:   P1D
    time_coverage_start:        1981-01-01T00:00:00
    time_standard:              Universal Time Coordinated (UTC)
    title:                      POWER Daily Meteorologging_Functiony Time Series
    version:                    POWER Version 9.0.1

In [5]:
kol_daily = xr.merge([ds_daily.sel(lat=22, lon=87.5)["CDD0"], ds_daily.sel(lat=22, lon=87.5)["CDD10"]])
kol_daily

<xarray.Dataset>
Dimensions:  (time: 16071)
Coordinates:
    lat      float64 22.0
    lon      float64 87.5
  * time     (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2024-12-31
Data variables:
    CDD0     (time) float64 dask.array<chunksize=(14610,), meta=np.ndarray>
    CDD10    (time) float64 dask.array<chunksize=(14610,), meta=np.ndarray>
Attributes:
    long_name:      Cooling Degree Days Above 0 C
    standard_name:  Cooling_Degree_Days_Above_0_C
    units:          degree-day

In [6]:
kol_daily_df = kol_daily.to_dataframe().reset_index(level=['time'])
print(kol_daily_df.shape)
print(kol_daily_df.memory_usage(deep=True).sum()/1024/1024, " MBs")

(16071, 5)
0.6131820678710938  MBs


In [10]:
s3_path = 'kolkata_901_daily_meteorology_utc.parquet'
local_path = '/Users/sumangangopadhyay/nasa-power-data-exploration/files_for_s3_upload/kolkata_901_daily_meteorology_utc.parquet'
kol_daily_df.to_parquet(local_path, index=0)

In [11]:
s3_client.upload_file(local_path, 'nasa-power-combined-data', s3_path)

In [12]:
bucket_contents = s3_client.list_objects_v2(Bucket='nasa-power-combined-data')
list_of_objs_in_bucket = bucket_contents['Contents']
for i in range(len(list_of_objs_in_bucket)):
    print(list_of_objs_in_bucket[i]['Key'])

kolkata_901_daily_meteorology_utc.parquet


In [13]:
download_path_local = '/Users/sumangangopadhyay/nasa-power-data-exploration/files_for_s3_upload/kolkata_901_daily_meteorology_utc_v2.parquet'
s3_client.download_file('nasa-power-combined-data', 'kolkata_901_daily_meteorology_utc.parquet', download_path_local)

In [14]:
df = pd.read_parquet(download_path_local)

In [15]:
df.shape

(16071, 5)

In [16]:
df.head(10)

,time,lat,lon,CDD0,CDD10
0,1981-01-01,22.0,87.5,18.9375,8.9375
1,1981-01-02,22.0,87.5,18.1250,8.1250
2,1981-01-03,22.0,87.5,17.9375,7.9375
3,1981-01-04,22.0,87.5,20.8125,10.8125
4,1981-01-05,22.0,87.5,21.4375,11.4375
5,1981-01-06,22.0,87.5,21.5625,11.5625
6,1981-01-07,22.0,87.5,18.8125,8.8125
7,1981-01-08,22.0,87.5,16.6875,6.6875
8,1981-01-09,22.0,87.5,15.5000,5.5000
9,1981-01-10,22.0,87.5,15.3750,5.3750
